In [2]:
import torch
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from utils import load_embeddings
import wandb

/home/ubuntu/anaconda3/envs/unitary_detoxify/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
wandb.init(entity="anitavero", project="finetune")
%env WANDB_SILENT=true

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anitavero. Use `wandb login --relogin` to force relogin


env: WANDB_SILENT=true


In [4]:
ids, embeddings, metadata = load_embeddings('/home/ubuntu/dev/detoxify/jigsaw-toxic-comment-classification-challenge/embeddings/embeddings_t5-large_test_This_text_is_about_{}.pkl')

In [5]:
labels = pd.read_csv('/home/ubuntu/dev/detoxify/jigsaw-toxic-comment-classification-challenge/data/test_labels.csv', dtype={"id": "string"})
classes = list(labels.columns)
classes.remove("id")
y = labels[classes].to_numpy()

In [6]:
mask = (y != -1).any(axis=1)
y_m = y[mask, :]
embeddings_m = embeddings[mask, :]
y_m.shape, embeddings_m.shape

X_train, X_test, y_train, y_test = train_test_split(embeddings_m, y_m, random_state=1)

In [7]:
clf = MLPClassifier(random_state=1, max_iter=100, verbose=True)
mo_clf = MultiOutputClassifier(estimator=clf).fit(X_train, y_train)

Iteration 1, loss = 0.22498170
Iteration 2, loss = 0.16746005
Iteration 3, loss = 0.15783564
Iteration 4, loss = 0.15290194
Iteration 5, loss = 0.14867438
Iteration 6, loss = 0.14538247
Iteration 7, loss = 0.14318401
Iteration 8, loss = 0.13958453
Iteration 9, loss = 0.13692006
Iteration 10, loss = 0.13452032
Iteration 11, loss = 0.13128942
Iteration 12, loss = 0.12774620
Iteration 13, loss = 0.12491148
Iteration 14, loss = 0.12226801
Iteration 15, loss = 0.11916301
Iteration 16, loss = 0.11563826
Iteration 17, loss = 0.11295411
Iteration 18, loss = 0.10987703
Iteration 19, loss = 0.10684017
Iteration 20, loss = 0.10471133
Iteration 21, loss = 0.10090460
Iteration 22, loss = 0.09811528
Iteration 23, loss = 0.09526682
Iteration 24, loss = 0.09283641
Iteration 25, loss = 0.08971750
Iteration 26, loss = 0.08722871
Iteration 27, loss = 0.08432123
Iteration 28, loss = 0.08162082
Iteration 29, loss = 0.07947609
Iteration 30, loss = 0.07703359
Iteration 31, loss = 0.07417719
Iteration 32, los

/home/ubuntu/anaconda3/envs/unitary_detoxify/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 0.05714955
Iteration 2, loss = 0.02208094
Iteration 3, loss = 0.01918709
Iteration 4, loss = 0.01779728
Iteration 5, loss = 0.01676192
Iteration 6, loss = 0.01597111
Iteration 7, loss = 0.01530683
Iteration 8, loss = 0.01480578
Iteration 9, loss = 0.01421314
Iteration 10, loss = 0.01368666
Iteration 11, loss = 0.01308836
Iteration 12, loss = 0.01259890
Iteration 13, loss = 0.01213982
Iteration 14, loss = 0.01164086
Iteration 15, loss = 0.01126325
Iteration 16, loss = 0.01058169
Iteration 17, loss = 0.01012415
Iteration 18, loss = 0.00963553
Iteration 19, loss = 0.00911546
Iteration 20, loss = 0.00853844
Iteration 21, loss = 0.00824575
Iteration 22, loss = 0.00780584
Iteration 23, loss = 0.00731816
Iteration 24, loss = 0.00687966
Iteration 25, loss = 0.00649606
Iteration 26, loss = 0.00610300
Iteration 27, loss = 0.00587533
Iteration 28, loss = 0.00534567
Iteration 29, loss = 0.00503042
Iteration 30, loss = 0.00454991
Iteration 31, loss = 0.00423114
Iteration 32, los

/home/ubuntu/anaconda3/envs/unitary_detoxify/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 0.04750812
Iteration 2, loss = 0.01651497
Iteration 3, loss = 0.01350375
Iteration 4, loss = 0.01161083
Iteration 5, loss = 0.01043928
Iteration 6, loss = 0.00959441
Iteration 7, loss = 0.00896649
Iteration 8, loss = 0.00833521
Iteration 9, loss = 0.00785745
Iteration 10, loss = 0.00735793
Iteration 11, loss = 0.00709400
Iteration 12, loss = 0.00667127
Iteration 13, loss = 0.00637211
Iteration 14, loss = 0.00594137
Iteration 15, loss = 0.00543760
Iteration 16, loss = 0.00511903
Iteration 17, loss = 0.00472784
Iteration 18, loss = 0.00439077
Iteration 19, loss = 0.00413484
Iteration 20, loss = 0.00385333
Iteration 21, loss = 0.00344824
Iteration 22, loss = 0.00330583
Iteration 23, loss = 0.00294729
Iteration 24, loss = 0.00263656
Iteration 25, loss = 0.00249522
Iteration 26, loss = 0.00231880
Iteration 27, loss = 0.00211768
Iteration 28, loss = 0.00196518
Iteration 29, loss = 0.00176852
Iteration 30, loss = 0.00168248
Iteration 31, loss = 0.00147736
Iteration 32, los

/home/ubuntu/anaconda3/envs/unitary_detoxify/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 0.07650579
Iteration 2, loss = 0.04063282
Iteration 3, loss = 0.03550833
Iteration 4, loss = 0.03281448
Iteration 5, loss = 0.03066304
Iteration 6, loss = 0.02911505
Iteration 7, loss = 0.02767295
Iteration 8, loss = 0.02640991
Iteration 9, loss = 0.02522935
Iteration 10, loss = 0.02395869
Iteration 11, loss = 0.02297139
Iteration 12, loss = 0.02203265
Iteration 13, loss = 0.02112568
Iteration 14, loss = 0.02055499
Iteration 15, loss = 0.01969023
Iteration 16, loss = 0.01891322
Iteration 17, loss = 0.01854548
Iteration 18, loss = 0.01755764
Iteration 19, loss = 0.01689975
Iteration 20, loss = 0.01618949
Iteration 21, loss = 0.01553711
Iteration 22, loss = 0.01461584
Iteration 23, loss = 0.01378362
Iteration 24, loss = 0.01322585
Iteration 25, loss = 0.01271620
Iteration 26, loss = 0.01181586
Iteration 27, loss = 0.01144396
Iteration 28, loss = 0.01054481
Iteration 29, loss = 0.01017965
Iteration 30, loss = 0.00967366
Iteration 31, loss = 0.00912046
Iteration 32, los

In [8]:
def predict_prob(embs):
    preds = mo_clf.predict_proba(embs)
    return np.column_stack([p[:, 1] for p in preds])

y_probs = predict_prob(X_test)
y_pred = mo_clf.predict(X_test)
score = mo_clf.score(X_test, y_test)

In [9]:
from model_eval.eval_predictions import evaluate

In [10]:
evaluate(y_probs, y_test)

{'auc_scores': [0.9332486899501415,
  0.9698520767649705,
  0.9364603929743527,
  0.9578011147486553,
  0.9391207122163853,
  0.9428209870325803],
 'mean_auc': 0.9465506622811809}

In [20]:
sweep_config = {
  "name" : "my-sweep",
  "method" : "bayes",
  "metric": {
    "name": "val_loss",
    "goal": "minimize"
  },
  "parameters" : {
    "epochs" : {
      "values" : [10, 20, 50]
    },
    "learning_rate" :{
      "min": 0.0001,
      "max": 0.1
    }
  }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: gwfebgo4
Sweep URL: https://wandb.ai/anitavero/uncategorized/sweeps/gwfebgo4


In [21]:
def train():
    config = wandb.config
    for epoch in range(config["epochs"]):
        loss = mo_clf.fit(X_train, y_train)  # your model training code here
        wandb.log({"loss": loss, "epoch": epoch})

In [22]:
count = 5 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 2v2wkzew with config:
wandb: 	epochs: 10
wandb: 	learning_rate: 0.017130624069023218
Exception in thread Thread-31 (_run_job):
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/unitary_detoxify/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 300, in _run_job
    self._function()
  File "/tmp/ipykernel_11812/2152270066.py", line 3, in train
  File "/home/ubuntu/anaconda3/envs/unitary_detoxify/lib/python3.10/site-packages/wandb/sdk/wandb_config.py", line 128, in __getitem__
    return self._items[key]
KeyError: 'epochs'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/unitary_detoxify/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/envs/unitary_detoxify/lib/python3.10/threading.py", line 953, in run
    self._t